# Previous Work

The previous net saved in _'01-07-17_convnet.h5'_ , gets a 97% accuracy rate on the test dataset (!) of GTSRB. However, when it must predict on images that don't belong to the GTSRB dataset, it sees traffic signs where there are none, and with very high (~100%) confidence. This is a problem because during a race, the car won't see any traffic signs most of the time.

The approach to fix this, is to add a _background_ class, also called _Zero_ class, as the 43th class to the dataset.

In [ ]:
#Load Picture data functions + Load Numpy and Plot library
from utilities.gtsrb_loader.load_data import load_data
from utilities.gtsrb_loader.get_folderpath import get_folderpath
import matplotlib.pyplot as plt
from keras.models import load_model
import numpy as np

In [ ]:
# Load Trainingsdata (could take 1 or 2 minutes to load data) / X --> images, y --> labels
path = get_folderpath(subset='train', original_images=True)
X_train, y_train = load_data(path)

In [ ]:
#Load zero class (Pictures with no traffic signs on them)
path_zero = "C:\\Users\\tomas\\Desktop\\GTSRB\\Zero_Class\\ 00043"
X_train_zero, y_train_zero = load_data(path_zero)
print (X_train_zero) #Frage: X_train_zero ist leer?

In [ ]:
#Resizing images to 64 x 64
from skimage.transform import resize
X_train_zero = np.array([resize(pic, (64, 64), mode='edge') for pic in X_train_zero])

# Training Set Selection

For this second net, we want to distinguish between _background_ and _traffic sign_ reliably. Thats why I dont balance number of training images per class yet. Any traffic sign does the job. Since our zero class is only in gray color, we will transfor the GTSRB data to grayscale

# Black and White Zero class

In [ ]:
#Generate Black and white pictures (First just for class 29 to test)
from skimage import color
x = color.rgb2gray(X_train[0])
print (y_train[0])
print ("Matrix x: (Anzahl Zeilen, Anzahl Spalten)", x.shape)

In [ ]:
#Show generated grey picture X_train[0]
plt.subplot(1, 2, 1)
plt.imshow(X_train[0])
plt.subplot(1, 2, 2)
plt.imshow(x, cmap='gray')
plt.show()

In [ ]:
#Generate Black and white pictures for all classes
X_train_gray = np.array([color.rgb2gray(pic) for pic in X_train])

In [ ]:
print (X_train_zero.shape, X_train_gray.shape) #Q: What is the purpose of this line?

In [ ]:
#Pictures get edited in Preprocessing to generate a bigger amount of data

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(  #Adjust datagenerator
    featurewise_center=True, #Set input mean to 0 over the dataset, feature-wise 
    #Q: What does this line mean?
    featurewise_std_normalization=True, #Divide inputs by std of the dataset, feature-wise. 
    #Q: What does this line mean?
    rotation_range=20, #Pictures get rotated (max. 20°)
    width_shift_range=0.2, #Pictures get shifted horizontally (max. by the factor 0.2)
    height_shift_range=0.2, #Pictures get shifted vertically (max. by the factor 0.2)
    horizontal_flip=True) #Pictures get mirrored horizontally 
    #Q: Is this executed with all signs?

In [ ]:
X_train_zero = X_train_zero.reshape(len(X_train_zero), 64, 64, 1) # grayscale wrapper for the datagen object 
print (X_train_zero.shape)

In [ ]:
datagen.fit(X_train_zero) #Compute the internal data stats related to the 
#data-dependent transformations, based on an array of sample data.
#Only required if featurewise_center or featurewise_std_normalization

In [ ]:
X_train_gray.max(), X_train_gray.min() #Frage: Was macht diese Funktion?

# Color zero class
since we have a colorfull zeroclass we do not need to transform the pictures to grey


In [ ]:
#Load colorful signs
#path = get_folderpath(subset='train', original_images=False)
path="/data_on_server/Images/GTSRB_64x64/Final_Training"
X_train, y_train = load_data(path)


In [ ]:
#Load colorful zero classes
#path_zero = "Y:\\Zero_Class_color\\00043"
path_zero = "//data_on_server//Images//Zero_Class_color"
X_train_zero, y_train_zero = load_data(path_zero)

In [ ]:
#The whole dataset consists of the zeroclasses as well as the normal signs
X_trainC=X_train+X_train_zero
Y_trainC=y_train+y_train_zero

In [ ]:
#List the trainingimages in a numpy array
sign=np.array(Y_trainC)
unique_classes = len(np.unique(sign))
y_trainN = np.eye(unique_classes)[np.array(sign, dtype=int)]
X_trainN = np.array(X_trainC, dtype=np.float32) / 255

In [ ]:
#Load Testdata:
path="/data_on_server/Images/GTSRB_64x64/Final_Test"
X_test, Y_test = load_data(path)

In [ ]:
#List the testimages in a numpy array
sign_test=np.array(Y_test)
unique_classes_test = len(np.unique(sign_test))+1
Y_testN = np.eye(unique_classes_test)[np.array(sign_test, dtype=int)]
X_testN = np.array(X_test, dtype=np.float32) / 255

In [ ]:
#Identify all Trainingimages which have not the specified format
for n in X_trainN:
    if(n.shape != (64,64,3)):
        print(n.shape)

In [ ]:
#Number of training images
print (np.array(y_trainN).shape)
print (X_trainN.shape)

In [ ]:
#grayscale wrapper for the datagen object #Q: What should be done here?
X_train = X_trainN.reshape(len(X_train), 64, 64, 3)
X_train.shape

# CHECKPOINT WIP

In [ ]:
# normalize X_train
X_train = np.array(X_train, dtype=np.float32) / 255
# X_train_zero

In [ ]:
# one-hot encode the labels
boundBox=np.array(y_train)[:,0:4]
sign=np.array(y_train)[:,4]
unique_classes = len(np.unique(sign))
y_trainN = np.eye(unique_classes)[np.array(sign, dtype=int)]
y_trainF=np.concatenate([y_trainN,boundBox], axis=1)

In [ ]:
y_trainF



## Define Network:
From Keras docs, VGG-like Convnet.

First Block (Convolutional Block)

In [ ]:
#Import Keras Model libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Conv3D, SeparableConv2D

In [ ]:
#Generate Sequential model
model = Sequential()

In [ ]:
#Add 2D convolution layer
model.add(SeparableConv2D(input_shape=(64, 64, 3), # Specifying the input shape
                 filters=32, #Number of filters
                 kernel_size=(3, 3), #Filtersize (3x3 filter)
                 activation='relu')) #Activation function is Rectifier

In [ ]:
#Add 2D convolution layer
model.add(Conv2D(filters=32,
                 kernel_size=(3, 3),
                 activation='relu'))

In [ ]:
#Add MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2))) #poolsize 2x2

In [ ]:
#Applies Dropout to the input. Dropout consists in randomly setting a fraction rate (here 0.25) of input units to 0
# at each update during training time, which helps prevent overfitting.
model.add(Dropout(0.25))

Second Block (Convolutional Block)

In [ ]:
#Add 2D convolution layer
model.add(Conv2D(filters=64, #Number of filters
                 kernel_size=(3, 3), #Filtersize (3x3 filter)
                 activation='relu')) #Activation function is Rectifier

In [ ]:
#Add 2D convolution layer
model.add(Conv2D(filters=64, #Number of filters
                 kernel_size=(3, 3), #Filtersize (3x3 filter)
                 activation='relu')) #Activation function is Rectifier

In [ ]:
#Add MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2))) #poolsize 2x2

In [ ]:
#Applies Dropout to the input. Dropout consists in randomly setting a fraction rate (here 0.25) of input units to 0
# at each update during training time, which helps prevent overfitting.
model.add(Dropout(0.25))

Third Block (Fully Connected Layers)

In [ ]:
#Flatten the input --> From 2D-Array to 1D-Array
model.add(Flatten())

In [ ]:
#Add fullyconnected layers
model.add(Dense(200, activation='relu')) #Add layer with 200 neurons with rectifier activation function
model.add(Dropout(0.5)) #Applies Dropout to the input --> fraction rate is here 0.5
model.add(Dense(44, activation='softmax')) #Add layer with 44 neurons with softmax activation function

Define critical parameters for network (SGD-optimizer, tensorboard)

In [ ]:
#Stochastic gradient descent optimizer.
#Includes support for momentum, learning rate decay, and Nesterov momentum.
from keras.optimizers import SGD
sgd = SGD(lr=0.01, #Learningrate = 0.01, 
          decay=1e-5, #decay (Factor to decrease the learningrate over time),
          momentum=0.9, #momentum (former gradient descents are considered to increase the speed of convergence)= 0.9
          nesterov=True) #Nesterov momentum is used (method to reduce the probability to not find minima because of high momentum)

In [ ]:
model.compile(loss='categorical_crossentropy', #As lossfunction y_true and y_pred are compared in every category
              optimizer=sgd, #Stochastic gradient descent optimizers are used
              metrics=['accuracy']) #List of metrics to be evaluated by the model during training and testing --> Here: accuracy.

In [ ]:
#Tensorboard
from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs', #Path of directory where to save the log files to be parsed by TensorBoard.
                          histogram_freq=0, #frequency (in epochs) at which to compute activation and weight histograms
                                            #If set to 0, histograms won't be computed.
                          write_graph=True, #Visualize the graph in TensorBoard.
                          write_images=False) #Write model weights to visualize as image in TensorBoard.

In [ ]:
#Other Callbacks: #Q: Where are they used?
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

#Q: What is this/ Is this training function used?
model.fit(X_trainN, y_trainN, batch_size=32, epochs=50, validation_data=(X_testN, Y_testN), shuffle=True, callbacks=[TestCallback((X_testN, Y_testN))])

OR load model:

In [ ]:
model = load_model('9-11-17_convnet2_un.h5')

for 3D-Conv Layer at start:

In [ ]:
length=len(X_trainN)
np.reshape(X_trainN,(length,1,64,64,3))
np.reshape(X_trainN,(length,1,64,64,3))


## Train Model

In [ ]:
#Train model with tensorborad
model.fit(X_trainN, y_trainN, batch_size=32, epochs=50, validation_data=(X_testN, Y_testN), shuffle=True, callbacks=[tensorboard])

In [ ]:
#Define model with TestCallback
#Q: Will this be done?

In [ ]:
#Training without tensorboard
model.fit(X_trainN, y_trainN, batch_size=32, verbose=2 ,epochs=20)
#Q: What is the benefit of training with and without tensorboard

In [ ]:
#Save model
model.save('9-11-17_convnet2v1.h5')